# AG2: Swarm (Deep-dive)

Deep-dive
- https://docs.ag2.ai/docs/user-guide/advanced-concepts/swarm/deep-dive

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Swarm Deep-dive

### Components

Here are the main components that are needed to create a swarm chat:
- **Create Agents**: instantiate an AssistantAgent to be part of the swarm chat.
  - API Reference: AssistantAgent
- **Register Handoffs**: utilize register_hand_off to register OnCondition, OnContextCondition, and AfterWork handoffs.
  - API Reference: register_hand_off, OnCondition, OnContextCondition, AfterWork, AfterWorkOption, SwarmResult
- **Update Agent State (Optional)**: update an agent’s state before replying.
  - API Reference: UpdateSystemMessage
- **Start Swarm Chat**: initiate the swarm chat with initiate_swarm_chat or a_initiate_swarm_chat for asynchronous calls.
  - API Reference: initiate_swarm_chat, a_initiate_swarm_chat

### Create Agents

### Registering Handoffs to agents

### Registering Context Variable-based Handoffs to agents

In [2]:
from autogen import AssistantAgent, OnCondition, register_hand_off

# llm_config = ...
llm_config = {"api_type": "openai", "model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}

agent_2 = AssistantAgent("agent_2", llm_config=llm_config)
agent_3 = AssistantAgent("agent_3", llm_config=llm_config)

# --------Option 1---------
agent_1 = AssistantAgent("agent_1", llm_config=llm_config)
# Register the handoff
register_hand_off(
    agent = agent_1,
    hand_to=[
        OnCondition(target=agent_2, condition="condition_1"),
        OnCondition(target=agent_3, condition="condition_2"),
    ]
)

# --------Option 2---------
# This is equivalent to:
def transfer_to_agent_2():
    """condition_1"""
    return agent_2

def transfer_to_agent_3():
    """condition_2"""
    return agent_3

agent_1 = AssistantAgent("agent_1", llm_config=llm_config, functions=[transfer_to_agent_2, transfer_to_agent_3])

In [3]:
from autogen import OnContextCondition, ContextExpression

register_hand_off(
    agent = agent_1,
    hand_to=[
        OnContextCondition(
            target=agent_2,
            condition=ContextExpression("(${account_level} > 2 and ${budget_remaining} > 0) or ${account_tier} == 'Gold' or len(${order_count}) > 10"),
            available="logged_in"),
        OnCondition(target=agent_3, condition="condition_2"), # LLM-based, evaluated after OnContextCondition's
    ]
)

### Enabling/Disabling Handoffs

In [4]:
register_hand_off(
    agent = agent_1,
    hand_to=[
        OnCondition(
            target=agent_2,
            condition="Transfer to the reviewer to evaluate the plan.",
            available="has_plan"),
    ]
)

### SwarmResult

### Update Agent state before replying

In [5]:
# # Creates a system message string
# def create_system_prompt_function(my_agent: ConversableAgent, messages: List[Dict[]]) -> str:
#  preferred_name = my_agent.get_context("preferred_name", "(name not provided)")

#     # Note that the returned string will be treated like an f-string using the context variables
#     return "You are a customer service representative helping a customer named "
#  + preferred_name
#  + " and their passport number is '{passport_number}'."

# # Function to update an Agent's state
# def my_callable_state_update_function(my_agent: ConversableAgent, messages: List[Dict[]]) -> None:
#  agent.set_context("context_key", 43)
#  agent.update_system_message("You are a customer service representative helping customer ID " + agent.get_context("context_key"))

# # Create the AssistantAgent and set agent updates
# customer_service = AssistantAgent(
#     name="CustomerServiceRep",
#     system_message="You are a customer service representative.",
#     update_agent_state_before_reply=[
#  UpdateSystemMessage("You are a customer service representative. Quote passport number '{passport_number}'"),
#  UpdateSystemMessage(create_system_prompt_function),
#  my_callable_state_update_function]
#  ...
# )

# -> ERROR: SyntaxError: invalid syntax. Perhaps you forgot a comma?

from typing import List, Dict
from autogen import ConversableAgent, AssistantAgent, UpdateSystemMessage

# Creates a system message string
def create_system_prompt_function(my_agent: ConversableAgent, messages: List[Dict]) -> str:
    preferred_name = my_agent.get_context("preferred_name", "(name not provided)")
    # Note that the returned string will be treated like an f-string using the context variables
    return "You are a customer service representative helping a customer named " + preferred_name + " and their passport number is '{passport_number}'."

# Function to update an Agent's state
def my_callable_state_update_function(my_agent: ConversableAgent, messages: List[Dict]) -> None:
    agent.set_context("context_key", 43)
    agent.update_system_message("You are a customer service representative helping customer ID " + agent.get_context("context_key"))

# Create the AssistantAgent and set agent updates
customer_service = AssistantAgent(
    name="CustomerServiceRep",
    system_message="You are a customer service representative.",
    update_agent_state_before_reply=[
        UpdateSystemMessage("You are a customer service representative. Quote passport number '{passport_number}'"),
        UpdateSystemMessage(create_system_prompt_function),
        my_callable_state_update_function
    ]
)

### Initialize SwarmChat with initiate_swarm_chat / a_initiate_swarm_chat

In [6]:
from autogen import initiate_swarm_chat

chat_history, context_variables, last_active_agent = initiate_swarm_chat(
    initial_agent=agent_1, # the first agent to start the chat
    agents=[agent_1, agent_2, agent_3], # a list of agents
    messages=[{"role": "user", "content": "Hello"}], # a list of messages to start the chat, you can also pass in one string
    # user_agent=user_agent, # optional, if you want to use your own user agent
    context_variables={"key": "value"} # optional, initial context variables
)

_User (to chat_manager):

Hello

--------------------------------------------------------------------------------

Next speaker: agent_1

agent_1 (to chat_manager):

Hello! How can I assist you today?

--------------------------------------------------------------------------------


### AfterWork

In [7]:
# # Register the handoff to an agent
# register_handoff(
#     agent=agent_1,
#     hand_to=[
#  OnCondition(...),
#  AfterWork(agent_4) # Fallback to agent_4 if no OnCondition handoff is called
#  ]
# )

# # Register the handoff to an AfterWorkOption
# register_handoff(
#     agent=agent_2,
#     hand_to=[AfterWork(AfterWorkOption.TERMINATE)] # Terminate the chat if no handoff is suggested
# )

# def my_after_work_func(last_speaker: AssistantAgent, messages: List[Dict[str, Any]], groupchat: GroupChat) -> Union[AfterWorkOption, AssistantAgent, str]:
#     if last_speaker.get_context("agent_1_done"):
#         return agent_2
#     else:
#         return AfterWorkOption.TERMINATE

# # Register the handoff to a function that will return an agent or AfterWorkOption
# agent_3.handoff(hand_to=[AfterWork(my_after_work_func)])
# register_handoff(agent_3, hand_to=[AfterWork(my_after_work_func)])

# # Register the swarm level AfterWork that becomes the default for agents that don't have one specified
# chat_history, context_variables, last_active_agent = initiate_swarm_chat(
#  ...
#     after_work=AfterWorkOption.TERMINATE # Or an agent or Callable
# )

# -> ERRORS -> REWRITE (REFACTOR) CODE

In [8]:
print(type(chat_history))
print(type(context_variables))
print(type(last_active_agent))

<class 'autogen.agentchat.chat.ChatResult'>
<class 'dict'>
<class 'autogen.agentchat.assistant_agent.AssistantAgent'>


In [9]:
print(len(chat_history.chat_history))
print(chat_history.summary)
print(chat_history.chat_history)

2
Hello! How can I assist you today?
[{'content': 'Hello', 'role': 'assistant'}, {'content': 'Hello! How can I assist you today?', 'name': 'agent_1', 'role': 'user'}]


In [10]:
print(context_variables)

{'key': 'value'}


In [11]:
print(last_active_agent.name)

agent_1


In [12]:
print(last_active_agent.system_message)

You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feed

### Q&As